### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

In [3]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

### Fusion

In [5]:
w_model = 0.4
w_efsr = 0.6
w_sum = w_model + w_efsr

In [6]:
confidence_model = np.where(
    inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [7]:
efsr_min = 0.20
efsr_max = 0.98

confidence_efsr = (radnet_input_efsr_edge - efsr_min) / (efsr_max - efsr_min)
confidence_efsr = np.clip(confidence_efsr, 0.0, 1.0)

In [8]:
fused = (w_model * confidence_model + w_efsr * confidence_efsr) / w_sum

In [9]:
cls = (fused >= 0.5).astype(int)
conf = np.where(cls == 1, fused, 1.0 - fused)

### Apply

In [10]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*62}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
--------------------------------------------------------------
-60    1.3   5      0.45   0.92   1        Bom      0.94      
-67    1.7   3      0.26   0.91   1        Bom      0.94      
-74    4.1   19     0.52   0.43   0        Ruim     0.80      
-64    1.3   15     0.50   0.88   1        Bom      0.90      
-60    3.4   70     0.78   0.50   0        Ruim     0.77      
-70    6.7   146    0.85   0.38   0        Ruim     0.86      
-63    4.9   35     0.83   0.41   0        Ruim     0.84      
-59    2.6   7      0.42   0.44   1        Bom      0.58      
-76    5.6   56     0.69   0.33   0        Ruim     0.90      
-61    1.5   13     0.22   0.95   1        Bom      0.97      
-52    9.7   31     0.75   0.88   1        Bom      0.52      
-83    7.9   119    0.29   0.33   0        Ruim     0.90      
-93    12.5  47     0.71   0.39   0        Ruim     0.85      
-65    2.8   26     0.70   0.93   1        Bom      0.6

### Save

In [11]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
